In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features
from src.visualization import visualize

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

DIAGS_RELEVANTES = [
    "C33X",
    "C340",
    "C341",
    "C342",
    "C343",
    "C381",
    "C384",
    "C450",
    "C780",
    "C782",
    "D143",
    "D381",
    "E848",
    "I080",
    "I081",
    "I340",
    "I350",
    "I351",
    "I352",
    "I420",
    "I456",
    "I472",
    "I495",
    "I710",
    "I712",
    "J398",
    "J47X",
    "J679",
    "J841",
    "J848",
    "J849",
    "J860",
    "J869",
    "J90X",
    "J91X",
    "J931",
    "J955",
    "J980",
    "M348",
    "Q211",
    "Q231",
    "Q676",
    "T820",
    "T821",
    "Z450",
]

DIAGS_FALTANTES_PULMON = ["C349", "C348"]

DIAGS_RELEVANTES += DIAGS_FALTANTES_PULMON

In [3]:
# Carga los egresos nacionales
egresos_nacionales = pd.read_csv(
    "../data/processed/ranking_nacional_egresos.csv", sep=";", encoding="latin-1"
)

# Filtra solamente los datos del INT
egresos_torax = egresos_nacionales.query("ESTABLECIMIENTO_SALUD == 112103").copy()

En primer lugar, se filtrarán ambas bases de datos para solamente tener la información de los diagnósticos más relevantes para el INT. Luego de esto, se calculará la cantidad de egresos, dias de estada y pacientes para cada uno de los diagnósticos. Estos insumos serán utilizados para estimar la demanda.


In [4]:
# Filtra solamente los diagnosticos mas relevantes del Torax
egresos_mas_relevantes_nacionales = (
    egresos_nacionales[egresos_nacionales["DIAG1"].isin(DIAGS_RELEVANTES)]
    .query("ANO_EGRESO >= 2017")
    .copy()
)

# Agrupa todos los diagnosticos relacionados a cancer de pulmon a un diagnostico comun
egresos_mas_relevantes_nacionales["DIAG1"] = egresos_mas_relevantes_nacionales["DIAG1"].replace(
    ["C340", "C341", "C342", "C343", "C348", "C349"], "C34"
)

# Filtra solamente los diagnosticos relevantes para el torax
egresos_mas_relevantes_torax = egresos_mas_relevantes_nacionales.query(
    "ESTABLECIMIENTO_SALUD == 112103"
).copy()

In [20]:
# Obtiene el resumen de metricas para el estrato nacional
metricas_diags_relevantes_nacionales = build_features.calculate_discharges_metrics(
    egresos_mas_relevantes_nacionales
)

# Obtiene resumen de metricas para el Torax
metricas_diags_relevantes_torax = build_features.calculate_discharges_metrics(
    egresos_mas_relevantes_torax
)

# Obtiene el porcentaje de resolucion de egresosx del Torax
porcentaje_egresos_nacionales_torax = (
    metricas_diags_relevantes_torax["n_egresos"] / metricas_diags_relevantes_nacionales["n_egresos"]
)

Una vez ya obtenido las metricas relevantes tanto para el estrato Nacional y del hospital a analizar, se procederá a comparar con los casos teóricos.


In [14]:
# Importa los casos teoricos por cada problema de salud
casos_teoricos = pd.read_excel("../data/interim/casos_teoricos_diagnosticos.xlsx")
casos_teoricos["Diagnostico"] = casos_teoricos["Diagnostico"].str.split("-").str[0].str.strip()
casos_teoricos = casos_teoricos.set_index("Diagnostico")

# Renombra columnas
casos_teoricos = casos_teoricos.rename(
    columns={
        "Poblacion 2017": 2017,
        "Poblacion 2018": 2018,
        "Poblacion 2019": 2019,
        "Poblacion 2020": 2020,
        "Poblacion 2021": 2021,
        "Poblacion 2022": 2022,
        "Poblacion 2023": 2023,
        "Poblacion 2024": 2024,
        "Poblacion 2025": 2025,
        "Poblacion 2026": 2026,
        "Poblacion 2027": 2027,
        "Poblacion 2028": 2028,
        "Poblacion 2029": 2029,
        "Poblacion 2030": 2030,
        "Poblacion 2031": 2031,
        "Poblacion 2032": 2032,
        "Poblacion 2033": 2033,
        "Poblacion 2034": 2034,
        "Poblacion 2035": 2035,
    }
)

# Obtiene los casos en el estrato nacional
casos_nacionales = casos_teoricos.query("Estrato == 'Pais' and Estadística == 'Incidencia'").copy()
casos_nacionales_del_2017_al_2020 = casos_nacionales[[i for i in range(2017, 2021)]]

Solamente se compararán los casos reales vs teóricos entre 2017 y 2020, ya que la información se encuentra disponible en tales periodos.

In [7]:
# Obtiene cuanto se hace cargo el INT de los casos teoricos
porcentaje_atencion_teorica_int = (
    metricas_diags_relevantes_torax["n_pacientes_distintos"].sum(axis=1)
    .div(casos_nacionales_del_2017_al_2020.sum(axis=1), axis=0)
)

In [8]:
cantidad_de_casos_int = casos_nacionales[[i for i in range(2017, 2036)]].mul(
    porcentaje_atencion_teorica_int, axis=0
)

In [9]:
metricas_diags_relevantes_torax

dias_estada_totales                   n_egresos                 \
                     2017  2018  2019  2020      2017 2018 2019 2020   
DIAG1                                                                  
C33X                    4    22    34    64         1    1    4    7   
C34                  4567  4547  4611  1846       358  408  403  151   
C381                  234   253   223    31        15   21   19    5   
C384                  144    62   159    17         9    9   10    3   
C450                  232   243   113    79        23   27    8    5   
C780                  458   514   450    97        57   63   62   15   
C782                  411   615   441   556        29   53   39   51   
D143                  465   373   184    49        32   40   28    6   
D381                  262   465   713   304        36   85   97   46   
E848                  377   602   574   366        26   35   41   23   
I080                  612   763   493   339        38   42   42   17   
I081                  511   981   797   242        29   44   42   14   
I340                  815   804   569   454        40   48   37   28   
I350                 1964  2084  1999  1126       118  163  148   83   
I351                  484   737   680   508        37   52   49   27   
I352                  324   446   476   369        31   26   37   21   
I420                  396   822   754   310        33   34   30   10   
I456                   82    27    63    16        44   25   29   16   
I472                  117   119   106   150        22   29   26   24   
I495                  173   148   267    70        51   64   91   23   
I710                  379   352   483   530        19   24   25   28   
I712                  165   139   170   341         8   13   15   29   
J398                  124    69   150   729        19   10    9   16   
J47X                  643   843   848   294        53   47   42   20   
J679                  864   789   969    94        77   70   69   10   
J841                 1564  1286  1799   930       108  108  122   58   
J848                  278   399   519   275        25   53   52   27   
J849                  343   245   233    87        43   23   27   10   
J860                  784   793    97   157        22   61   11   10   
J869                 1181   520   408   322        39   27   23   18   
J90X                  538   685   536   281        54   72   57   27   
J91X                    0    10     0     0         0    1    0    0   
J931                  417   266   226    70        21   30   19    5   
J955                  609   225   451   214        53   35   44   22   
J980                   40    73     7    54         4    6    2    7   
M348                  290   357   421    63        21   29   27    2   
Q211                  170    97   131   122        47   32   38   22   
Q231                    1     6    18   263         1    1    2   19   
Q676                   46    69    33    24         4   10    5    4   
T820                  174   156   348   475        10   17   25   26   
T821                  458   201   199   112        40   28   35   18   
Z450                  141   243   190    78        71  132  137   57   

      n_pacientes_distintos                dias_estada_promedio             \
                       2017 2018 2019 2020                 2017       2018   
DIAG1                                                                        
C33X                      1    1    4    4             4.000000  22.000000   
C34                     290  319  324  140            12.756983  11.144608   
C381                     13   18   19    4            15.600000  12.047619   
C384                      9    8    9    3            16.000000   6.888889   
C450                     17   18    7    5            10.086957   9.000000   
C780                     52   54   52   15             8.035088   8.158730   
C782                     27   48   34   48            14.172414

In [10]:
cantidad_de_casos_int

,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
C33X,2.432022,2.475887,2.522867,2.569224,2.598280,2.618112,2.635584,2.652153,2.668073,2.683370,2.698024,2.712004,2.725282,2.737833,2.749863,2.761203,2.771839,2.781763,2.790976
C34,260.955964,265.662629,270.703621,275.677785,278.795410,280.923386,282.798129,284.575994,286.284268,287.925643,289.498021,290.998017,292.422769,293.769528,295.060311,296.277039,297.418283,298.483192,299.471708
C381,13.132919,13.369787,13.623481,13.873812,14.030710,14.137803,14.232152,14.321625,14.407596,14.490200,14.569332,14.644821,14.716523,14.784301,14.849261,14.910494,14.967929,15.021521,15.071270
C384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C450,11.430504,11.636667,11.857475,12.075355,12.211915,12.305125,12.387243,12.465118,12.539945,12.611841,12.680715,12.746418,12.808826,12.867817,12.924357,12.977652,13.027641,13.074287,13.117586
C780,42.073981,42.832838,43.645598,44.447583,44.950239,45.293332,45.595598,45.882243,46.157669,46.422308,46.675823,46.917667,47.147380,47.364518,47.572632,47.768805,47.952808,48.124503,48.283882
C782,38.182746,38.871419,39.609011,40.336824,40.792991,41.104354,41.378664,41.638799,41.888751,42.128915,42.358984,42.578461,42.786929,42.983985,43.172851,43.350881,43.517866,43.673682,43.818321
D143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E848,19.855932,19.448861,19.642530,20.052676,20.288205,20.316739,20.248054,20.136995,19.995610,19.828680,19.640565,19.435876,19.218885,18.992924,18.909460,18.724334,18.537927,18.354596,18.179295
